# Code and Documentation
### Part 1 - Split data
____
#### GroupD: Nicole Guobadia, Dhruvi Kaswala, Arashpreet Singh Pandher, Karen Angella Edy Setiawan

Note: make sure to shut down all other kernels

Download packages

In [1]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
# BiocManager::install(version = "3.14")

Bioconductor version '3.10' is out-of-date; the current release version '3.16'
  is available with R version '4.2'; see https://bioconductor.org/install



In [2]:
install.packages('glmnet', dependencies=TRUE)
library(glmnet)


  There is a binary version available but the source version is later:
       binary source needs_compilation
glmnet  4.1-1  4.1-6              TRUE

  Binaries will be installed
package 'glmnet' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'glmnet'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\nicol\Anaconda3\Lib\R\library\00LOCK\glmnet\libs\x64\glmnet.dll to C:\Users\nicol\Anaconda3\Lib\R\library\glmnet\libs\x64\glmnet.dll: Permission denied"
Warning message:
"restored 'glmnet'"



The downloaded binary packages are in
	C:\Users\nicol\AppData\Local\Temp\RtmpsrduTg\downloaded_packages


Warning message:
"package 'glmnet' was built under R version 3.6.3"
Loading required package: Matrix

Loaded glmnet 4.1-1



Load dataset 2 into the notebook.

In [4]:
load(path.expand("AML_datasets.RData"))
ls()

[1] "data.1" "data.2" "data.3" "info.1" "info.2" "info.3"

In [5]:
set.seed(1)

In [6]:
# gene expression data
dim(data.2)
data.2[1:5, 1:8]

[1] 12708  8348

,GSM1030962_10030.CEL,GSM1030963_10110.CEL,GSM1030964_10111.CEL,GSM1030965_10138.CEL,GSM1030966_1032.CEL,GSM1030967_1113.CEL,GSM1030968_1132.CEL,GSM1030969_1446.CEL
PAX8,8.27154,8.49112,8.34082,8.24190,7.75355,7.95457,7.99231,8.40749
CCL5,5.02307,3.95575,4.68666,5.78429,4.97396,5.75343,4.21497,4.55103
MMP14,7.06640,6.41582,6.14716,6.65615,6.35464,6.89049,6.28378,6.63393
DTX2P1-UPK3BP1-PMS2P11,8.24990,8.42649,8.09809,8.15517,8.00680,7.95516,8.13866,8.56563
BAD,5.75432,6.77953,6.35168,4.97367,6.79648,5.49744,7.27109,6.34759


In [7]:
# phenotype data
dim(info.2)
info.2[1:5, ]

[1] 8348    8

,Dataset,GSE,Condition,Disease,Tissue,FAB,Filename,FAB_all
GSM1030962_10030.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030962_10030.CEL,unknown
GSM1030963_10110.CEL,2,GSE42038,CONTROL,ALL,PBMC,unknown,GSM1030963_10110.CEL,unknown
GSM1030964_10111.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030964_10111.CEL,unknown
GSM1030965_10138.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030965_10138.CEL,unknown
GSM1030966_1032.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030966_1032.CEL,unknown


For our model, we only look for samples with AML and ALL. Therefore, we subset dataset 2 with only AML and ALL samples, 5086 samples in total. 

In [8]:
# AML samples
length(which(info.2['Disease'] == 'AML' ))

[1] 2588

In [9]:
# ALL samples
length(which(info.2['Disease'] == 'ALL' ))

[1] 2498

In [10]:
# subset of data containing only AML and ALL samples
leukemia <- c('AML', 'ALL')
subset.info <- subset(info.2, Disease %in% leukemia)

In [11]:
# phenotype data (AML and ALL only)
dim(subset.info)
subset.info[1:5,]

[1] 5086    8

,Dataset,GSE,Condition,Disease,Tissue,FAB,Filename,FAB_all
GSM1030962_10030.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030962_10030.CEL,unknown
GSM1030963_10110.CEL,2,GSE42038,CONTROL,ALL,PBMC,unknown,GSM1030963_10110.CEL,unknown
GSM1030964_10111.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030964_10111.CEL,unknown
GSM1030965_10138.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030965_10138.CEL,unknown
GSM1030966_1032.CEL,2,GSE42038,CONTROL,ALL,BM,unknown,GSM1030966_1032.CEL,unknown


In [12]:
# gene expression data (AML and ALL only)
subset.data.2 <- data.2[,rownames(subset.info)]
dim(subset.data.2)
subset.data.2[1:10,1:10]

[1] 12708  5086

,GSM1030962_10030.CEL,GSM1030963_10110.CEL,GSM1030964_10111.CEL,GSM1030965_10138.CEL,GSM1030966_1032.CEL,GSM1030967_1113.CEL,GSM1030968_1132.CEL,GSM1030969_1446.CEL,GSM1030970_1509.CEL,GSM1030971_1524.CEL
PAX8,8.27154,8.49112,8.34082,8.24190,7.75355,7.95457,7.99231,8.40749,8.11578,8.18438
CCL5,5.02307,3.95575,4.68666,5.78429,4.97396,5.75343,4.21497,4.55103,9.11314,7.51884
MMP14,7.06640,6.41582,6.14716,6.65615,6.35464,6.89049,6.28378,6.63393,6.55267,6.79243
DTX2P1-UPK3BP1-PMS2P11,8.24990,8.42649,8.09809,8.15517,8.00680,7.95516,8.13866,8.56563,7.87479,7.76304
BAD,5.75432,6.77953,6.35168,4.97367,6.79648,5.49744,7.27109,6.34759,5.37534,4.98609
PRPF8,9.40864,9.99876,10.10850,9.31322,9.89784,9.68242,9.99797,10.66240,9.52855,9.17292
CAPNS1,10.40220,11.13020,10.90360,10.16460,10.91150,10.20030,10.12300,10.75990,9.81339,10.01970
RPL35,12.97710,12.90460,12.92400,13.22060,12.94350,12.60770,13.13370,13.46040,12.74550,12.78510
EIF4G2,4.88702,5.13691,4.90962,5.02729,5.90961,6.75055,5.12968,4.58441,4.72627,5.89014
EIF3D,10.75580,10.91820,10.50810,11.10610,11.08930,10.44550,11.26500,10.95390,10.03550,11.15850


To build our models, we divide our data into training set and test set, consisting of 80% and 20% of our dataset correspondingly. Our training set contains 4021 samples and our test set contains 1065 samples.

In [13]:
subset.data.2 <- as.data.frame(t(subset.data.2))

# add class label
subset.data.2$label <- with(subset.info, ifelse(Disease=='AML', 1, 0))

# reference: https://www.geeksforgeeks.org/split-the-dataset-into-the-training-test-set-in-r/
# divide the matrix into training set 80% and testing 20% respectively with replacement
sample <- sample(c(TRUE,FALSE), nrow(subset.data.2), replace=TRUE, prob=c(0.8,0.2))
  
# creating training dataset
train_dataset <- subset.data.2[sample, ]
  
# creating testing dataset
test_dataset <- subset.data.2[!sample,]

In [14]:
dim(train_dataset)
dim(test_dataset)

[1]  4021 12709

[1]  1065 12709

In each training and test, we further check if there is the class label, AML and ALL, are balanced.

In [15]:
# making sure that the data will be balanced between AML and ALL
length(which(train_dataset$label == 1))
length(which(train_dataset$label == 0))

[1] 2037

[1] 1984

In [16]:
# making sure that the data will be balanced between AML and ALL
length(which(test_dataset$label == 1))
length(which(test_dataset$label == 0))

[1] 551

[1] 514

In [17]:
train_dataset[1:5, 1:5]

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD
GSM1030962_10030.CEL,8.27154,5.02307,7.06640,8.24990,5.75432
GSM1030963_10110.CEL,8.49112,3.95575,6.41582,8.42649,6.77953
GSM1030964_10111.CEL,8.34082,4.68666,6.14716,8.09809,6.35168
GSM1030966_1032.CEL,7.75355,4.97396,6.35464,8.00680,6.79648
GSM1030969_1446.CEL,8.40749,4.55103,6.63393,8.56563,6.34759


Saved our training and test set into RData so it can be loaded to the next ipynb file. 

In [18]:
# saving train and test data so it can be loaded to another ipynb file
save(train_dataset, test_dataset, file = "data.RData")